In [1]:
import os
from functools import partial

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression


In [2]:
os.chdir('..')
os.getcwd().split('/')[-1] == 'GeneralLoss'

True

In [3]:
from minimizers import links
from minimizers import losses
from minimizers.minimize import GeneralLossMinimizer, CustomLossRegressor

In [4]:
seed = 0

In [5]:
X, y = make_regression(
    n_targets=1, 
    n_samples=1000, 
    n_features=10, 
    random_state=seed,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
del X, y

---
### Mean Squared Error

In [6]:
model = Pipeline([
    ('sc', StandardScaler()),
    ('est', GeneralLossMinimizer(
        loss_fn=losses.mse_loss,
        link_fn=links.linear_link,
        random_state=seed,
    )),
])
model[-1].set_estimator_type('regressor')
model[-1].set_multi_output(False)

model.fit(X_train, y_train)
y_hat = model.predict(X_train)
print('Training error:', ((y_train-y_hat)**2).mean()**0.5)

y_hat = model.predict(X_test)
print('Training error:', ((y_test-y_hat)**2).mean()**0.5)

Training error: 1.79766868199184e-05
Training error: 1.9921911612004604e-05


In [7]:
model = Pipeline([
    ('sc', StandardScaler()),
    ('est', CustomLossRegressor(
        loss_fn=losses.multi_mse_loss,
        link_fn=links.linear_link,
        random_state=seed,
    )),
])

model.fit(X_train, y_train)
y_hat = model.predict(X_train)
print('Training error:', ((y_train-y_hat)**2).mean()**0.5)

y_hat = model.predict(X_test)
print('Training error:', ((y_test-y_hat)**2).mean()**0.5)

Training error: 4.9928711244253936e-05
Training error: 5.395171147134042e-05


---
### Pseudo-Huber Loss

In [8]:
delta = 1.

model = Pipeline([
    ('sc', StandardScaler()),
    ('est', GeneralLossMinimizer(
        loss_fn=partial(losses.pseudo_huber_loss, delta=delta),
        link_fn=links.linear_link,
        random_state=seed,
    )),
])
model[-1].set_estimator_type('regressor')
model[-1].set_multi_output(False)

model.fit(X_train, y_train)
y_hat = model.predict(X_train)
print('Training error:', ((y_train-y_hat)**2).mean()**0.5)

y_hat = model.predict(X_test)
print('Training error:', ((y_test-y_hat)**2).mean()**0.5)

Training error: 1.489636803489353e-05
Training error: 1.5202140865817542e-05


In [9]:
delta = 1.

model = Pipeline([
    ('sc', StandardScaler()),
    ('est', CustomLossRegressor(
        loss_fn=partial(losses.multi_pseudo_huber_loss, delta=delta),
        link_fn=links.linear_link,
        random_state=seed,
    )),
])

model.fit(X_train, y_train)
y_hat = model.predict(X_train)
print('Training error:', ((y_train-y_hat)**2).mean()**0.5)

y_hat = model.predict(X_test)
print('Training error:', ((y_test-y_hat)**2).mean()**0.5)

Training error: 1.489636803489353e-05
Training error: 1.5202140865817542e-05


---
### Gaussian MLE

In [10]:
model = Pipeline([
    ('sc', StandardScaler()),
    ('est', GeneralLossMinimizer(
        loss_fn=losses.gaussian_mle,
        link_fn=links.linear_link,
        random_state=seed,
    )),
])
model[-1].set_estimator_type('regressor')
model[-1].set_multi_output(False)

model.fit(X_train, y_train)
y_hat = model.predict(X_train)
print('Training error:', ((y_train-y_hat)**2).mean()**0.5)

y_hat = model.predict(X_test)
print('Training error:', ((y_test-y_hat)**2).mean()**0.5)

Training error: 1.797324097906514e-05
Training error: 1.991772088218421e-05


In [11]:
model = Pipeline([
    ('sc', StandardScaler()),
    ('est', CustomLossRegressor(
        loss_fn=losses.multivariate_gaussian_mle,
        link_fn=links.linear_link,
        random_state=seed,
    )),
])

model.fit(X_train, y_train)
y_hat = model.predict(X_train)
print('Training error:', ((y_train-y_hat)**2).mean()**0.5)

y_hat = model.predict(X_test)
print('Training error:', ((y_test-y_hat)**2).mean()**0.5)

Training error: 1.797324097906514e-05
Training error: 1.991772088218421e-05


---
### Poisson MLE

In [12]:
model = Pipeline([
    ('sc', StandardScaler()),
    ('est', GeneralLossMinimizer(
        loss_fn=losses.poisson_mle,
        link_fn=links.log_link,
        random_state=seed,
    )),
])
model[-1].set_estimator_type('regressor')
model[-1].set_multi_output(False)

yp_train = np.exp(y_train*0.01).round()
yp_test = np.exp(y_test*0.01).round()
model.fit(X_train, yp_train)
y_hat = model.predict(X_train)
print('Training error:', ((yp_train-y_hat)**2).mean()**0.5)

y_hat = model.predict(X_test)
print('Training error:', ((yp_test-y_hat)**2).mean()**0.5)

Training error: 0.3523098506065447
Training error: 0.6084790052573759


---